In [ ]:
import numpy as np
from pysheds.grid import Grid
from matplotlib import pyplot as plt
import seaborn as sns
import logging
import os

In [ ]:
####### Parameters ##############

#dem Filename
file_name="toulon2bis.tif"
# Il faut que le système de projection associé au fichier .tif soit au format WGS84 (et non pas Lambert-93)

#accumulation value
acc_value=200

# innondation max level
innondation_max_level=20

#logging level
log_level=logging.DEBUG 

# Save files
save_file=True

#folder 
folder="Debug"

In [ ]:
#### INIT #######
def init(folder):
    # Check whether the specified path exists or not
    isExist = os.path.exists(folder)
    if not isExist:
    # Create a new directory because it does not exist
        os.makedirs(folder)

In [ ]:
###### PLOTING FONCTIONS ##########
def plot_height_map(hand_view,grid,save_file=False,folder=""):
    #plot
    fig, ax = plt.subplots(figsize=(8,6))
    fig.patch.set_alpha(0)
    plt.imshow(hand_view, 
            extent=grid.extent, cmap='terrain', zorder=1)
    plt.colorbar(label='Height above nearest drainage (m)')
    plt.grid(zorder=0)
    plt.title('HAND', size=14)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.tight_layout()
    if not save_file:
        plt.show()
    else :
        plt.savefig(os.path.join(folder,"height.jpg"))
    
    
    
    
def plot_elevation_map(dem,grid,save_file=False,folder=""):
    fig, ax = plt.subplots(figsize=(8,6))
    fig.patch.set_alpha(0)
    plt.imshow(dem, extent=grid.extent, cmap='terrain', zorder=1)
    plt.colorbar(label='Elevation (m)')
    plt.grid(zorder=0)
    plt.title('Digital elevation map', size=14)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.tight_layout()
    
    


def plot_innondation(dem,grid,inundation_extent,level,save_file=False,folder=""):
    fig, ax = plt.subplots(figsize=(8,6))
    fig.patch.set_alpha(0)
    dem_view = grid.view(dem, nodata=np.nan)
    plt.imshow(dem_view, extent=grid.extent, cmap='Greys', zorder=1)
    plt.imshow(inundation_extent, extent=grid.extent,
            cmap='Blues', vmin=-5, vmax=10, zorder=2)
    plt.grid(zorder=0)
    plt.title('Inundation depths (constant channel depth)', size=14)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.tight_layout()
    if not save_file:
        plt.show()
    else :
        plt.savefig(os.path.join(folder,"inondation"+str(i)+".jpg"))

In [ ]:
###### INIT #######
# set logging  
init(folder)
logging.basicConfig(filename="log.log", encoding='utf-8', level=log_level)

# Instantiate grid from raster
grid = Grid.from_raster(file_name)
dem = grid.read_raster(file_name)


# Resolve flats, pits, depressions and compute flow directions
pit_filled_dem = grid.fill_pits(dem)
flooded_dem = grid.fill_depressions(pit_filled_dem)
inflated_dem = grid.resolve_flats(flooded_dem)
fdir = grid.flowdir(inflated_dem)


# Compute accumulation
# Le paramètre accumulation permet de mesurer le nombre de pixels (représentant des gouttes d'eau) qui viennent
# s'accumuler à certains endroits du terrain. Ce qui permet de mettre en évidence des cours d'eau où ces pixels
# viennent s'accumuler.
acc = grid.accumulation(fdir)

# Compute height above nearest drainage
# Le paramètre hand permet de mesurer la différence d'altitude entre un point situé sur le cours d'eau
# et un point situé sur la rive à l'extérieur de celui-ci.
hand = grid.compute_hand(fdir, dem, acc > 2000) # on choisit les cours d'eau où il y a plus de 2000 pixels
                                                # qui se sont accumulés.

# Create a view of HAND in the catchment
# Cette commande fait en sorte que les différences d'altitude calculées précédemment puisse être affichées sur une
# carte.
hand_view = grid.view(hand, nodata=np.nan)




# Cette partie sert à délimiter une partie de la carte pour se focaliser sur un fleuve précis par exemple
# en renseignant la latitude et la longitude du point le plus extrême

'''# Delineate a catchment
catch = grid.catchment(x=latitude, y=longitude, fdir=fdir, xytype='coordinate')'''

'''# Clip to the catchment
grid.clip_to(catch)'''

# On affiche la topographie de la zone considérée (fichier .tif)
plot_elevation_map(dem,grid,save_file,folder)

# On affiche la carte avec les différences d'altitude
plot_height_map(hand_view,grid,save_file,folder)


# On affiche 20 cartes avec les 20 niveaux d'innondation considérés (paramètre "innondation_max_level=20"
#                                                                                     définit auparavant)
for i in range(innondation_max_level):
    inundation_extent = np.where(hand_view<i,i-hand_view, np.nan)
    plot_innondation(dem,grid,inundation_extent,i,save_file,folder)

In [ ]:
# Calcul des points extrèmes de l'image avec les niveaux d'inondations pour pouvoir l'afficher sur une carte
# interactive de type Folium

from shapely.geometry import Point
import geopandas as gpd
d = {'geometry': [Point(grid.extent[0], grid.extent[2]), Point(grid.extent[1], grid.extent[3])]}
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
# gdf = gdf.to_crs(4326)
grid_tuple = tuple([gdf["geometry"][0].xy[0][0],gdf["geometry"][1].xy[0][0],gdf["geometry"][0].xy[1][0],gdf["geometry"][1].xy[1][0]])
gdf

In [ ]:
# On définit les images concernant les niveaux d'inondation correspondant aux scénarios décennal, centennal et
#                                                                                                      millénal
# Ces images seront affichées en différentes couleurs sur la carte de type folium suivante.
# Différentes valeurs du paramètre "hand_view" sont utilisées pour modélisr la crue des scénarios
#                                                                   décennal, centennal et millénal:

# Scénario décennal: 0<hand_view<2
# Scénario centennal: 3<hand_view<5
# Scénario millénal: 5<hand_view<8


image_dec = np.where((hand_view<2) & (hand_view>0),2-hand_view, 0)
image_cent = np.where((hand_view<5) & (hand_view>3),5-hand_view, 0)
image_mill = np.where((hand_view<8) & (hand_view>5),8-hand_view, 0)

In [ ]:
import folium

m = folium.Map([(grid_tuple[2]+grid_tuple[3])/2,(grid_tuple[0]+grid_tuple[1])/2], tiles='CartoDB dark_matter', zoom_start=12)

folium.raster_layers.ImageOverlay(
    image=image_mill,
    name='Décennal (Scénario projetté)',
    bounds=[[grid_tuple[2],grid_tuple[0]], [grid_tuple[3],grid_tuple[1]]],
    origin='upper',
    colormap=lambda x: (0, 0, 1, x),
).add_to(m)

folium.raster_layers.ImageOverlay(
    image=image_cent,
    name='Centennal (Scénario projetté)',
    bounds=[[grid_tuple[2],grid_tuple[0]], [grid_tuple[3],grid_tuple[1]]],
    origin='upper',
    colormap=lambda x: (0, 1, 1, x),
).add_to(m)

folium.raster_layers.ImageOverlay(
    image=image_dec,
    name='Millénal (Scénario projetté)',
    bounds=[[grid_tuple[2],grid_tuple[0]], [grid_tuple[3],grid_tuple[1]]],
    origin='upper',
    colormap=lambda x: (1, 1, 1, x),
).add_to(m)




c_10_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#EFF3FF',
    'fillOpacity': 0.7
}

c_10 = folium.GeoJson(
    data=(open("Data_Dec.json", 'r').read()),
    name="Décénale",
    style_function=lambda x: c_10_bordersStyle)

c_10.add_to(m)



c_100_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#6AAED4',
    'fillOpacity': 0.7
}

c_100 = folium.GeoJson(
    data=(open("Data_Cent.json", 'r').read()),
    name="Centenale",
    style_function=lambda x: c_100_bordersStyle)

c_100.add_to(m)



c_1000_bordersStyle = {
    'color': 'black',
    'weight': 1,
    'fillColor': '#06519A',
    'fillOpacity': 0.7
}

c_1000 = folium.GeoJson(
    data=(open("Data_Mill.json", 'r').read()),
    name="Millénale",
    style_function=lambda x: c_1000_bordersStyle)

c_1000.add_to(m)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid', 'cours_deau']).add_to(c_10)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid', 'cours_deau']).add_to(c_100)

# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['ht_min', 'ht_max', 'uuid', 'cours_deau']).add_to(c_1000)

'''# and finally adding a tooltip/hover to the geojson file
folium.GeoJsonTooltip(['NOM_M']).add_to(c_EAIP)'''

folium.LayerControl().add_to(m)

In [ ]:
# Voir le site https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe
# pour des explications sur cette partie avec du code HTML

# We import the required library:
from branca.element import Template, MacroElement # branca: version 0.6.0

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Périodes de retour</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#06519A;opacity:1;'></span>1000 ans (TRI)</li>
    <li><span style='background:#6AAED4;opacity:1;'></span>100 ans (TRI)</li>
    <li><span style='background:#EFF3FF;opacity:1;'></span>10 ans (TRI)</li>
  </ul>
</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>

{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m

In [ ]:
# Exporting the map in HTML format

m.save("Carte_Inondations_Toulon_Scénarios_Projetté_Pysheds.html")